In [24]:
#sys path
from sys import path
from pathlib import Path

module_path = str(Path.cwd().parents[1])

if module_path not in path:
    path.append(module_path)
    
path.append(module_path + '\\functions')


# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import pandapower as pp
import pandapower.networks as net
import pandapower.topology as top
import pandapower.plotting as plot
import pandapower.converter
import pandapower.estimation

import time

pd.set_option('display.max_columns', 100)

import julia
julia.install()

import save_outputs

# Power System - case9

In [25]:
net = pandapower.networks.case9()
net

This pandapower network includes the following parameter tables:
   - bus (9 elements)
   - load (3 elements)
   - gen (2 elements)
   - ext_grid (1 element)
   - line (9 elements)
   - poly_cost (3 elements)
   - bus_geodata (9 elements)
 and the following results tables:
   - res_line (9 elements)
   - res_ext_grid (1 element)
   - res_load (3 elements)
   - res_gen (2 elements)

In [26]:
net.bus

,in_service,max_vm_pu,min_vm_pu,name,type,vn_kv,zone
0,True,1.1,0.9,1,b,345.0,1.0
1,True,1.1,0.9,2,b,345.0,1.0
2,True,1.1,0.9,3,b,345.0,1.0
3,True,1.1,0.9,4,b,345.0,1.0
4,True,1.1,0.9,5,b,345.0,1.0
5,True,1.1,0.9,6,b,345.0,1.0
6,True,1.1,0.9,7,b,345.0,1.0
7,True,1.1,0.9,8,b,345.0,1.0
8,True,1.1,0.9,9,b,345.0,1.0


In [27]:
net.load

,bus,const_i_percent,const_z_percent,controllable,in_service,name,p_mw,q_mvar,scaling,sn_mva,type
0,4,0.0,0.0,False,True,None,90.0,30.0,1.0,NaN,None
1,6,0.0,0.0,False,True,None,100.0,35.0,1.0,NaN,None
2,8,0.0,0.0,False,True,None,125.0,50.0,1.0,NaN,None


In [28]:
pp.create_sgen(net, bus=3, p_mw=50, q_mvar=15, controllable=False)
pp.create_sgen(net, bus=5, p_mw=30, q_mvar=10, controllable=False)

net.sgen

,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,controllable
0,None,3,50.0,15.0,NaN,1.0,True,None,True,False
1,None,5,30.0,10.0,NaN,1.0,True,None,True,False


In [29]:
net.gen

,bus,controllable,in_service,name,p_mw,scaling,sn_mva,type,vm_pu,slack,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar
0,1,True,True,None,163.0,1.0,NaN,None,1.0,False,300.0,10.0,300.0,-300.0
1,2,True,True,None,85.0,1.0,NaN,None,1.0,False,270.0,10.0,300.0,-300.0


In [30]:
net.ext_grid 

,bus,in_service,name,va_degree,vm_pu,max_p_mw,min_p_mw,max_q_mvar,min_q_mvar
0,0,True,None,0.0,1.0,250.0,10.0,300.0,-300.0


In [31]:
net.line

,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,0.000000,1.0,0,0.0,True,1.0,0.418370,100.0,None,1,0.000000,None,3,ol,68.558400
1,352.117636,1.0,3,0.0,True,1.0,0.418370,100.0,None,1,20.234250,None,4,ol,109.503000
2,797.836164,1.0,4,0.0,True,1.0,0.251022,100.0,None,1,46.419750,None,5,ol,202.342500
3,0.000000,1.0,2,0.0,True,1.0,0.502044,100.0,None,1,0.000000,None,5,ol,69.748650
4,465.775861,1.0,5,0.0,True,1.0,0.251022,100.0,None,1,14.163975,None,6,ol,119.977200
5,332.060303,1.0,6,0.0,True,1.0,0.418370,100.0,None,1,10.117125,None,7,ol,85.698000
6,0.000000,1.0,7,0.0,True,1.0,0.418370,100.0,None,1,0.000000,None,1,ol,74.390625
7,681.949347,1.0,7,0.0,True,1.0,0.418370,100.0,None,1,38.088000,None,8,ol,191.630250
8,392.232304,1.0,8,0.0,True,1.0,0.418370,100.0,None,1,11.902500,None,3,ol,101.171250


In [32]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0.0,ext_grid,150.0,5.0,0.1100,0.0,0.0,0.0
1,0.0,gen,600.0,1.2,0.0850,0.0,0.0,0.0
2,1.0,gen,335.0,1.0,0.1225,0.0,0.0,0.0


# OPF

In [33]:
input_dict = save_outputs.load_model('inputs\IEEE9_TA1')

x_d = np.array(input_dict['xd'])
xd_std = np.array(input_dict['xd_std'])
u = np.array(input_dict['u'])
alpha = np.array(input_dict['alpha'])
X_train = np.array(input_dict['x_train'])
Y_train = np.array(input_dict['y_train'])

In [34]:
X_d = x_d
Xd_std = xd_std
P_cont = u

In [35]:
# Load base
L_P_base = net.load['p_mw']
L_Q_base = net.load['q_mvar']

# RS base
RS_P_base = net.sgen['p_mw']
RS_Q_base = net.sgen['q_mvar']

# Load factor
faktor_load = L_Q_base/L_P_base
# RS factor
faktor_rs = RS_Q_base/RS_P_base

sum_base = np.sum(L_P_base) - np.sum(RS_P_base)

In [23]:
Pg_0 = []
Pg_1 = []
Pg_2 = []

np.random.seed(123)

count = P_cont.shape[0]

t = -time.time()
t2=0
for j in range(count):
    P_rand = []
    n_sim=100
    
    for k in range(X_d.shape[1]):
        rand = np.random.normal(X_d[j, k], Xd_std[j, k], n_sim).tolist()
        P_rand.append(rand)
        
    P_rand = np.array(P_rand)

    for i in range(n_sim):
        net.load['p_mw'] = pd.Series(P_rand[:3, i])
        net.sgen['p_mw'] = pd.Series(P_rand[3:, i])
        
        net.load['q_mvar'] = pd.Series(net.load['p_mw'] * faktor_load)
        net.sgen['q_mvar'] = pd.Series(net.sgen['p_mw'] * faktor_rs)
        
        sum_load = net.load['p_mw'].sum() - net.sgen['p_mw'].sum() 
        
        t1 = -time.time()
        pp.runopp(net)
        t1 += time.time()
        
        t2 += t1 

        Pg_0.append(net.res_ext_grid['p_mw'][0])
        Pg_1.append(net.res_gen['p_mw'][0])
        Pg_2.append(net.res_gen['p_mw'][1])
            
t += time.time()
print('Time of solver:', t)
print('Time of solver:', t2)

Time of solver: 64.38802671432495
Time of solver: 64.20859169960022


In [24]:
Pg_0_mean = np.array(Pg_0).mean()
Pg_1_mean = np.array(Pg_1).mean()
Pg_2_mean = np.array(Pg_2).mean()

print('Pg0:', Pg_0_mean)
print('Pg1:', Pg_1_mean)
print('Pg2:', Pg_2_mean)

Pg0: 63.76201553830014
Pg1: 102.79775696523795
Pg2: 71.8715462160191
